In [ ]:
!pip install fuzzywuzzy[speedup] python-Levenshtein openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm
import time
from google.colab import files
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl import Workbook
from openpyxl.drawing.image import Image
import matplotlib.pyplot as plt

# Función para cargar los archivos CSV
def cargar_archivos():
    uploaded = files.upload()
    nombres_archivos = list(uploaded.keys())
    print("Archivos cargados:", nombres_archivos)

    clientes = pd.read_csv(nombres_archivos[0])
    base_datos = pd.read_csv(nombres_archivos[1])

    return clientes, base_datos

# Función para comparar registros
def comparar_registros(clientes, base_datos):
    clientes['Similitud'] = 0
    clientes['Encontrado'] = 'No'
    clientes['Registro_BBDD'] = ''
    clientes['Sugerencia'] = ''

    start_time = time.time()

    for index, cliente in tqdm(clientes.iterrows(), total=clientes.shape[0]):
        nombre_apellido_cliente = f"{cliente['Nombre']} {cliente['Apellido']}"
        mejor_coincidencia = process.extractOne(nombre_apellido_cliente, base_datos['Nombre'] + ' ' + base_datos['Apellido'], scorer=fuzz.token_sort_ratio)

        if mejor_coincidencia[1] >= 80:
            clientes.at[index, 'Similitud'] = mejor_coincidencia[1]
            clientes.at[index, 'Encontrado'] = 'Sí'
            clientes.at[index, 'Registro_BBDD'] = base_datos.loc[base_datos['Nombre'] + ' ' + base_datos['Apellido'] == mejor_coincidencia[0], 'Especialidad'].values[0]
        elif 65 <= mejor_coincidencia[1] < 80:
            clientes.at[index, 'Similitud'] = mejor_coincidencia[1]
            clientes.at[index, 'Sugerencia'] = 'Revisar posible coincidencia'

    total_registros = clientes.shape[0]
    total_encontrados = (clientes['Encontrado'] == 'Sí').sum()
    total_no_encontrados = total_registros - total_encontrados

    porcentaje_encontrados = (total_encontrados / total_registros) * 100
    porcentaje_no_encontrados = (total_no_encontrados / total_registros) * 100
    tiempo_total = time.time() - start_time

    return (clientes, total_registros, total_encontrados, total_no_encontrados,
            round(porcentaje_encontrados, 2), round(porcentaje_no_encontrados, 2),
            int(tiempo_total // 60), round(tiempo_total % 60, 2))

# Función para generar el informe
def generar_informe(clientes, total_registros, total_encontrados, total_no_encontrados, porcentaje_encontrados, porcentaje_no_encontrados, tiempo_min, tiempo_seg):
    # Crear el libro y hoja de trabajo
    wb = Workbook()
    ws = wb.active
    ws.title = "Informe"

    # Color de fondo
    fondo_color = '0C7BC0'

    # Insertar el título con la palabra IQVIA
    ws.merge_cells('A1:D1')  # Celdas para el título
    ws['A1'] = 'Puenteo Realizado - IQVIA'

    # Formatear el título
    ws['A1'].font = Font(size=20, bold=True, color='FFFFFF')
    ws['A1'].alignment = Alignment(horizontal="center", vertical="center")

    # Aplicar color de fondo a las celdas combinadas (A1:D1)
    fill_color_titulo = PatternFill(start_color=fondo_color, end_color=fondo_color, fill_type='solid')
    for row in ws['A1:D1']:
        for cell in row:
            cell.fill = fill_color_titulo

    # Preparar los encabezados con el fondo de color corporativo
    encabezados = ['Nombre', 'Apellido', 'Similitud', 'Encontrado', 'Registro_BBDD', 'Sugerencia']
    fill_color = PatternFill(start_color=fondo_color, end_color=fondo_color, fill_type='solid')

    # Escribir los encabezados en la fila 3, después del título
    for col_num, encabezado in enumerate(encabezados, 1):
        celda = ws.cell(row=3, column=col_num, value=encabezado)
        celda.fill = fill_color
        celda.font = Font(bold=True, color='FFFFFF')
        celda.alignment = Alignment(horizontal="center", vertical="center")

    # Insertar los datos de "clientes" debajo de los encabezados
    for r in dataframe_to_rows(clientes[['Nombre', 'Apellido', 'Similitud', 'Encontrado', 'Registro_BBDD', 'Sugerencia']], index=False, header=False):
        ws.append(r)

    # Agregar resumen al final
    ws.append([])  # Línea vacía
    ws.append(['Total Registros Procesados', total_registros])
    ws.append(['Total Registros Encontrados', total_encontrados])
    ws.append(['Total Registros No Encontrados', total_no_encontrados])
    ws.append(['Porcentaje Encontrados', f"{porcentaje_encontrados}%"])
    ws.append(['Porcentaje No Encontrados', f"{porcentaje_no_encontrados}%"])
    ws.append(['Tiempo Total', f"{tiempo_min} minutos y {tiempo_seg} segundos"])

    # Crear el gráfico de torta
    etiquetas = ['Encontrados', 'No Encontrados']
    datos = [porcentaje_encontrados, porcentaje_no_encontrados]
    colores = ['#0C7BC0', '#87CEEB']  # Colores: fondo del código y más claro

    plt.figure(figsize=(5, 5))
    plt.pie(datos, labels=etiquetas, autopct='%1.1f%%', startangle=90, colors=colores, wedgeprops={'edgecolor': 'black'})
    plt.title('Distribución de registros encontrados vs no encontrados')

    # Guardar el gráfico como imagen
    nombre_grafico = 'grafico_torta.png'
    plt.savefig(nombre_grafico)
    plt.close()

    # Insertar la imagen en el Excel
    img = Image(nombre_grafico)
    ws.add_image(img, 'F1')  # Ubicar la imagen en la celda F1 (puedes ajustar la ubicación)

    # Guardar el archivo como Excel
    informe_nombre = 'Puenteo_IQVIA.xlsx'
    wb.save(informe_nombre)
    print(f"Informe guardado como {informe_nombre}. Procediendo a descargar...")
    files.download(informe_nombre)

# Función principal
def main():
    clientes, base_datos = cargar_archivos()
    resultados = comparar_registros(clientes, base_datos)
    generar_informe(*resultados)

# Ejecutar la función principal
main()


Saving doc_cliente.csv to doc_cliente.csv
Saving doc_main.csv to doc_main.csv
Archivos cargados: ['doc_cliente.csv', 'doc_main.csv']


100%|██████████| 24/24 [00:00<00:00, 528.11it/s]


Informe guardado como informe_corporativo_con_grafico.xlsx. Procediendo a descargar...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>